# make dataset from label_csv

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import os
import cv2
from PIL import ImageFile, ImageOps, Image

In [41]:
orig_df = pd.read_csv("../dataset/label.csv")

In [42]:
without_normal_df = orig_df[(orig_df["label_id"] == "심재성 2도") | (orig_df["label_id"] == "표재성 2도") | (orig_df["label_id"] ==  "1도") | (orig_df["label_id"] == "3도")]

In [43]:
X_train, X_valid = train_test_split(without_normal_df, test_size=0.2, stratify=without_normal_df["label_id"])

In [44]:
os.mkdir("./images")
os.mkdir("./labels")

In [45]:
os.mkdir("./images/train")
os.mkdir("./images/val")
os.mkdir("./labels/train")
os.mkdir("./labels/val")

In [46]:
enum = {"심재성 2도": 2, "1도": 0, "3도" : 3, "표재성 2도": 1}

In [47]:
df = X_train
path = "train"

for fn in df["file_name"].unique():
    parse_df = df[df["file_name"]==fn]
    label_path = "./labels/" + path + "/" + fn.split(".")[0] + ".txt"
    label_txt = open(label_path, "w")
    label_txt.close()

    label_txt = open(label_path, "a")
    for i in range(parse_df.shape[0]):
        f = parse_df.iloc[i]
        f_name = f["file_name"]
        img_path = f"../dataset/raw/{f_name}"
        img = Image.open(img_path).convert("RGB")
        img = ImageOps.exif_transpose(img) # 화상 이미지 회전 이슈 
        
        height = (f["y2"]-f["y1"]) / img.size[1]
        width = (f["x2"]-f["x1"]) / img.size[0]
        center_y = (f["y1"] + (f["y2"]-f["y1"]) / 2) / img.size[1]
        center_x = (f["x1"] + (f["x2"]-f["x1"]) / 2) / img.size[0]

        class_id = enum[f["label_id"]]
        line = f"{class_id} {center_x} {center_y} {width} {height}\n"
        label_txt.write(line)
    
    shutil.copy(f"../dataset/raw/{fn}", f"./images/{path}")
    label_txt.close()

In [48]:
df = X_valid
path = "val"

for fn in df["file_name"].unique():
    parse_df = df[df["file_name"]==fn]
    label_path = "./labels/" + path + "/" + fn.split(".")[0] + ".txt"
    label_txt = open(label_path, "w")
    label_txt.close()

    label_txt = open(label_path, "a")
    for i in range(parse_df.shape[0]):
        f = parse_df.iloc[i]
        f_name = f["file_name"]
        img_path = f"../dataset/raw/{f_name}"
        img = Image.open(img_path).convert("RGB")
        img = ImageOps.exif_transpose(img) # 화상 이미지 회전 이슈 
        
        height = (f["y2"]-f["y1"]) / img.size[1]
        width = (f["x2"]-f["x1"]) / img.size[0]
        center_y = (f["y1"] + (f["y2"]-f["y1"]) / 2) / img.size[1]
        center_x = (f["x1"] + (f["x2"]-f["x1"]) / 2) / img.size[0]

        class_id = enum[f["label_id"]]
        line = f"{class_id} {center_x} {center_y} {width} {height}\n"
        label_txt.write(line)
    
    shutil.copy(f"../dataset/raw/{fn}", f"./images/{path}")
    label_txt.close()

# make dataset from unprocessed

In [21]:
imgs = os.listdir("../dataset/unprocessed/")

In [22]:
x1, y1, x2, y2, name = imgs[0].split('_')
x1, y1, x2, y2 = map(float, [x1, y1, x2, y2])

In [23]:
df = pd.DataFrame(columns=["file_name", "x1", "y1", "x2", "y2", "label_id"])

In [24]:
for i in range(len(imgs)):
    img = imgs[i]
    x1, y1, x2, y2, name = img.split('_')
    x1, y1, x2, y2 = map(float, [x1, y1, x2, y2])
    df.loc[i] = {"file_name": img, "x1": x1, "y1": y1, "x2": x2, "y2": y2, "label_id": 0}

In [25]:
df.to_csv("unprocessed.csv", index=False)

In [37]:
os.mkdir("../dataset/YOLO_dataset/hoya_set/images")
os.mkdir("../dataset/YOLO_dataset/hoya_set/labels")
os.mkdir("../dataset/YOLO_dataset/hoya_set/images/train")
os.mkdir("../dataset/YOLO_dataset/hoya_set/images/val")
os.mkdir("../dataset/YOLO_dataset/hoya_set/labels/train")
os.mkdir("../dataset/YOLO_dataset/hoya_set/labels/val")

In [27]:
X_train, X_valid = train_test_split(df, test_size=0.2)

In [38]:
df = X_train
path = "../dataset/YOLO_dataset/hoya_set/"
category = "train"

for fn in df["file_name"].unique():
    parse_df = df[df["file_name"]==fn]
    label_path = path + "labels/" + category + "/" + fn[:-4] + ".txt"
    label_txt = open(label_path, "w")
    label_txt.close()

    label_txt = open(label_path, "a")
    for i in range(parse_df.shape[0]):
        f = parse_df.iloc[i]
        f_name = f["file_name"]
        img_path = f"../dataset/unprocessed/{f_name}"
        img = Image.open(img_path).convert("RGB")
        img = ImageOps.exif_transpose(img) # 화상 이미지 회전 이슈 
        
        height = (f["y2"]-f["y1"]) / img.size[1]
        width = (f["x2"]-f["x1"]) / img.size[0]
        center_y = (f["y1"] + (f["y2"]-f["y1"]) / 2) / img.size[1]
        center_x = (f["x1"] + (f["x2"]-f["x1"]) / 2) / img.size[0]

        class_id = f["label_id"]
        line = f"{class_id} {center_x} {center_y} {width} {height}\n"
        label_txt.write(line)
    
    shutil.copy(f"../dataset/unprocessed/{fn}", f"{path}images/{category}")
    label_txt.close()

In [39]:
df = X_valid
path = "../dataset/YOLO_dataset/hoya_set/"
category = "val"

for fn in df["file_name"].unique():
    parse_df = df[df["file_name"]==fn]
    label_path = path + "labels/" + category + "/" + fn[:-4] + ".txt"
    label_txt = open(label_path, "w")
    label_txt.close()

    label_txt = open(label_path, "a")
    for i in range(parse_df.shape[0]):
        f = parse_df.iloc[i]
        f_name = f["file_name"]
        img_path = f"../dataset/unprocessed/{f_name}"
        img = Image.open(img_path).convert("RGB")
        img = ImageOps.exif_transpose(img) # 화상 이미지 회전 이슈 
        
        height = (f["y2"]-f["y1"]) / img.size[1]
        width = (f["x2"]-f["x1"]) / img.size[0]
        center_y = (f["y1"] + (f["y2"]-f["y1"]) / 2) / img.size[1]
        center_x = (f["x1"] + (f["x2"]-f["x1"]) / 2) / img.size[0]

        class_id = f["label_id"]
        line = f"{class_id} {center_x} {center_y} {width} {height}\n"
        label_txt.write(line)
    
    shutil.copy(f"../dataset/unprocessed/{fn}", f"{path}images/{category}")
    label_txt.close()